In [1]:
import sys
import os
import math
import pandas as pd
import numpy as np
import json
from math import sqrt
from collections import defaultdict
# Python visualisation library
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Another much simpler viz library
import seaborn as sns

# Modelling Libaries
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
import sklearn.metrics
from surprise import Reader, Dataset, SVD, evaluate
from collections import defaultdict
from surprise.model_selection import KFold, train_test_split
#import recmetrics
from sklearn.model_selection import cross_validate as cv
from sklearn.model_selection import train_test_split

# Adding root of git repo to path so we can import pyfireanalytics
sys.path.append('..')

# Display formatting options
pd.set_option('display.max_columns', 1000)

# Load Data

In [2]:
data = pd.read_csv('~/desktop/movielens/UDATA.csv', encoding="latin-1")
item = pd.read_csv('~/desktop/movielens/UITEM.csv', encoding="latin-1")
genre = pd.read_csv('~/desktop/movielens/UGENRE.csv', encoding="latin-1")
user = pd.read_csv('~/desktop/movielens/UUSER.csv', encoding="latin-1")
test = pd.read_csv('~/desktop/movielens/U1TEST.csv', encoding="latin-1")
train = pd.read_csv('~/desktop/movielens/U1BASE.csv', encoding="latin-1")

test = pd.read_csv('~/desktop/movielens/test1.csv', encoding="latin-1")

# Data Cleaning

In [3]:
# rename itemid to movieid to join data and item together
data.rename(columns={'itemid':'movieid'}, inplace=True)
test.rename(columns={'itemid':'movieid'}, inplace=True)
train.rename(columns={'itemid':'movieid'}, inplace=True)
#join
df = pd.merge(data, item, on='movieid', how='outer')
df = pd.merge(df, user, on='userid', how='outer')

user2 = pd.merge(user, data, on='userid', how='inner')
user2 = user2.drop('timestamp', 1)
user2 = user2.drop_duplicates()
#change unix timestamp to readable date
df['reviewtimestamp'] = pd.to_datetime(df['timestamp'],unit='s')

#Split reviewtimestamp into year month day
df['reviewyear'] = df['reviewtimestamp'].dt.year
df['reviewmonth'] = df['reviewtimestamp'].dt.month
df['reviewday'] = df['reviewtimestamp'].dt.day

In [4]:
df.head()

,userid,movieid,rating,timestamp,movietitle,releasedate,imdburl,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,filmnoir,horror,musical,mystery,romance,scifi,thriller,war,western,age,gender,occupation,zipcode,reviewtimestamp,reviewyear,reviewmonth,reviewday
0,196,242,3,881250949,Kolya (1996),24-Jan-97,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,49,M,writer,55105,1997-12-04 15:55:49,1997,12,4
1,196,257,2,881251577,Men in Black (1997),4-Jul-97,http://us.imdb.com/M/title-exact?Men+in+Black+...,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,49,M,writer,55105,1997-12-04 16:06:17,1997,12,4
2,196,111,4,881251793,"Truth About Cats & Dogs, The (1996)",26-Apr-96,http://us.imdb.com/M/title-exact?Truth%20About...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,49,M,writer,55105,1997-12-04 16:09:53,1997,12,4
3,196,25,4,881251955,"Birdcage, The (1996)",8-Mar-96,"http://us.imdb.com/M/title-exact?Birdcage,%20T...",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,49,M,writer,55105,1997-12-04 16:12:35,1997,12,4
4,196,382,4,881251843,"Adventures of Priscilla, Queen of the Desert, ...",1-Jan-94,http://us.imdb.com/M/title-exact?Adventures%20...,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,49,M,writer,55105,1997-12-04 16:10:43,1997,12,4


# CF test 2

In [56]:
#number of unique users and movies
#test['userid'] = test['userid'].fillna(0)
#test['movieid'] = test['movieid'].fillna(0)
#test['rating'] = test['rating'].fillna(data['rating'].mean())

#n_users = data.userid.unique().shape[0]
#n_movies = data.movieid.unique().shape[0]

#user-item matrix to calculate similarity
#matrix = np.zeros((n_users, n_movies))
#for line in data.itertuples():
 #   matrix[line[1]-1, line[2]-1] = line[3]

    #--------------------------
#data = data.sample(frac=0.2)
#train_data, test_data = train_test_split(data, test_size=0.2)

#train_matrix = train_data.as_matrix(columns = ['userid', 'movieid', 'rating'])
#test_matrix = test_data.as_matrix(columns = ['userid', 'movieid', 'rating'])

#calculate similarity
#user_similarity = 1 - pairwise_distances(train_data, metric='cosine')
#user_similarity[np.isnan(user_similarity)] = 0
#movie_similarity = 1 - pairwise_distances(train_matrix.T, metric='cosine')
#movie_similarity[np.isnan(movie_similarity)] = 0

/Users/kennyliang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/kennyliang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [57]:
#print(train_matrix.shape)
#print(test_matrix.shape)

(16000, 3)
(4000, 3)


In [58]:
#prediction function
# np.newaxis is used so that mean_user_rating has same format as ratings
#def predict(ratings, similarity, type='user'):
#    if type =='user':
#        mean_user_rating = ratings.mean(axis=1)
#        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
#        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
#    elif type =='movie':
#        pred = ratings.dot(similarity) /np.array([np.abs(similarity).sum(axis=1)])
#    return pred

In [61]:
##def rmse(pred, actual):
  #  pred = pred[actual.nonzero()].flatten()
 #   actual = actual[actual.nonzero()].flatten()
#    return sqrt(mean_squared_error(pred,actual))

In [62]:
#user_prediction = predict(train_matrix, user_similarity, type='user')
#movie_prediction = predict(train_matrix, movie_similarity, type='movie')

In [63]:
#Test
#print('User-Based Collaborative Filtering RMSE: ' + str(rmse(user_prediction, test_matrix)))
#print('Item-Based Collaborative Filtering RMSE: ' + str(rmse(movie_prediction, test_matrix)))

User-Based Collaborative Filtering RMSE: 284.34155754859415
Item-Based Collaborative Filtering RMSE: 344.6053679721419


In [126]:
#Train
#print('User-Based Collaborative Filtering RMSE: ' + str(rmse(user_prediction, train_matrix)))
#print('Item-Based Collaborative Filtering RMSE: ' + str(rmse(movie_prediction, train_matrix)))

User-Based Collaborative Filtering RMSE: 199.83083046729354
Item-Based Collaborative Filtering RMSE: 260.8307482230392


# CF recommendations

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [6]:

Ratings_import = pd.read_csv('~/desktop/movielens/UDATA.csv', encoding="latin-1")
movies= pd.read_csv('~/desktop/movielens/UITEM.csv', encoding="latin-1")
movies = movies.rename(columns={'movieid':'itemid'})

Ratings_import['date'] = pd.to_datetime(Ratings_import['timestamp'],unit='s').apply(lambda x: x.date()).astype(str)

def group(Ratings_import):
    if ((Ratings_import['date'] >= '1997-09-01') & (Ratings_import['date'] < '1998-02-01')):
        return 'train'
    elif ((Ratings_import['date'] >= '1998-02-01') & (Ratings_import['date'] < '1998-04-30')):
        return 'test'

Ratings_import['group'] = Ratings_import.apply(group, axis=1)

Ratings = Ratings_import[Ratings_import['group'] == 'train']

Mean = Ratings.groupby(by="userid",as_index=False)['rating'].mean()
Rating_avg = pd.merge(Ratings,Mean,on='userid')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head()

,userid,itemid,rating_x,timestamp,date,group,rating_y,adg_rating
0,196,242,3,881250949,1997-12-04,train,3.615385,-0.615385
1,196,393,4,881251863,1997-12-04,train,3.615385,0.384615
2,196,381,4,881251728,1997-12-04,train,3.615385,0.384615
3,196,251,3,881251274,1997-12-04,train,3.615385,-0.615385
4,196,655,5,881251793,1997-12-04,train,3.615385,1.384615


In [29]:
check = pd.pivot_table(Rating_avg, values='rating_x', index='userid', columns='itemid')
final = pd.pivot_table(Rating_avg, values='adg_rating', index='userid', columns='itemid')
final.head()

itemid,1,10,100,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,102,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,103,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,104,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,105,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,106,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,107,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,108,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,109,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,11,110,1100,1101,1102,1103,1104,1105,1107,1108,1109,111,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,112,1120,1121,1122,1123,1124,1125,1126,1127,1128,1129,113,1130,1131,1132,1133,1134,1135,1136,1137,1138,1139,114,1140,1141,1142,1143,1144,1145,1146,1147,1148,1149,115,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,116,1160,1161,1162,1163,1164,1165,1166,1167,1168,1169,117,1170,1171,1172,1173,1174,1175,1176,1177,1178,1179,118,1180,1181,1182,1183,1184,1185,1186,1187,1188,1189,119,1190,1192,1193,1194,1195,1197,1198,1199,12,120,1200,1202,1203,1204,1205,1206,1207,1208,1209,121,1210,1211,1212,1213,1214,1215,1216,1217,1218,1219,122,1220,1221,1222,1223,1224,1225,1226,1227,1228,1229,123,1230,1231,1232,1233,1234,1238,1239,124,1240,1241,1242,1244,1245,1246,1247,1248,1249,125,1250,1251,1252,1253,1254,1255,1256,1257,1258,1259,126,1260,1261,1262,1263,1264,1265,1266,1267,1268,1269,127,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279,128,1280,1281,1282,1283,1284,1285,1286,1287,1288,1289,129,1290,1291,1292,1293,1295,1296,1297,1298,1299,13,130,1300,1301,1302,1303,1305,1306,1307,1308,131,1311,1312,1314,1315,1316,1317,1318,1319,132,1320,1321,1322,1323,1324,1325,1326,1327,1328,1329,133,1330,1331,1332,1333,1334,1335,1336,1337,1338,1339,134,1340,1341,1342,1343,1344,1345,1346,1347,1348,1349,135,1350,1351,1352,1353,1354,1355,1356,1357,1358,1359,136,1360,1361,1362,1363,1364,1365,1366,1367,1368,1369,137,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379,138,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,139,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,14,140,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,141,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,142,1420,1421,1422,1423,1424,1425,1426,1427,1428,1429,143,1432,1434,1435,1436,1437,1438,1439,144,1440,1441,1442,1443,1444,1445,1446,1449,145,1450,1451,1454,1455,1456,1459,146,1462,1463,1464,1465,1466,1467,1468,1469,147,...,531,532,533,534,535,536,537,538,539,54,540,541,542,543,544,545,546,547,548,549,55,550,551,552,553,554,555,556,557,558,559,56,560,561,562,563,564,565,566,567,568,569,57,570,571,572,573,574,575,576,577,578,579,58,580,581,582,583,584,585,586,587,588,589,59,590,591,592,593,594,595,596,597,598,6,60,601,602,603,604,605,606,607,608,609,61,610,611,612,613,614,615,616,617,618,619,62,620,621,622,623,624,625,626,627,628,629,63,630,631,632,633,634,635,636,637,638,639,64,640,641,642,643,644,645,646,647,648,649,65,650,651,652,653,654,655,656,657,658,659,66,660,661,662,663,664,665,666,667,668,669,67,670,671,672,673,674,675,676,678,679,68,680,681,682,683,684,685,686,687,688,689,69,690,692,693,694,695,696,697,698,699,7,70,700,701,702,703,704,705,706,707,708,709,71,710,711,712,713,714,715,716,717,718,719,72,720,721,722,723,724,725,726,727,728,729,73,730,731,732,733,734,735,736,737,738,739,74,740,741,742,743,744,745,746,747,748,749,75,750,751,753,754,755,756,757,758,759,76,760,761,762,763,764,765,766,767,768,769,77,770,771,772,773,774,775,776,777,778,779,78,780,781,782,783,784,785,786,787,788,789,79,790,791,792,793,794,795,796,797,798,799,8,80,800,801,802,803,804,805,806,807,808,809,81,810,811,812,813,815,816,817,818,819,82,820,821,822,823,824,825,826,827,828,829,83,830,831,832,833,834,835,836,837,838,839,84,840,841,842,843,844,845,846,847,848,849,85,850,851,852,853,854,855,856,857,858,859,86,860,861,862,863,864,865,866,867,868,869,87,870,871,872,873,874,875,876,877,878,879,88,880,881,882,883,884,885,886,887,888,889,89,890,892,893,894,895,896,

In [8]:
# Replace nulls with averages 
final_movie = final.fillna(final.mean(axis=0))

In [9]:
final_movie.head()

itemid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,356,357,358,359,360,361,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,...,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128,1129,1130,1131,1132,1133,1134,1135,1136,1137,1138,1139,1140,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1168,1169,1170,1171,1172,1173,1174,1175,1176,1177,1178,1179,1180,1181,1182,1183,1184,1185,1186,1187,1188,1189,1190,1192,1193,1194,1195,1197,1198,1199,1200,1202,1203,1204,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214,1215,1216,1217,1218,1219,1220,1221,1222,1223,1224,1225,1226,1227,1228,1229,1230,1231,1232,1233,1234,1238,1239,1240,1241,1242,1244,1245,1246,1247,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257,1258,1259,1260,1261,1262,1263,1264,1265,1266,1267,1268,1269,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280,1281,1282,1283,1284,1285,1286,1287,1288,1289,1290,1291,1292,1293,1295,1296,1297,1298,1299,1300,1301,1302,1303,1305,1306,1307,1308,1311,1312,1314,1315,1316,1317,1318,1319,1320,1321,1322,1323,1324,1325,1326,1327,1328,1329,1330,1331,1332,1333,1334,1335,1336,1337,1338,1339,1340,1341,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1366,1367,1368,1369,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427,1428,1429,1432,1434,1435,1436,1437,1438,1439,1440,1441,1442,1443,1444,1445,1446,1449,1450,1451,1454,1455,1456,1459,1462,1463,1464,1465,1466,1467,1468,1469,1470,1471,1472,1473,1474,1475,1476,1477,1478,1479,1480,1481,1482,1484,1485,1486,1487,1488,1489,1490,1491,1495,1496,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1517,1518,1519,1520,1521,1522,1523,1524,1528,1529,1530,1531,1534,1535,1536,1537,1539,1540,1541,1542,1543,1544,1545,1546,15

In [10]:
# user similarity on replacing NAN by user avg
#b = cosine_similarity(final_user)
#np.fill_diagonal(b, 0 )
#similarity_with_user = pd.DataFrame(b,index=final_user.index)
#similarity_with_user.columns=final_user.index
#similarity_with_user.head()

In [16]:
# user similarity on replacing NAN by item(movie) avg
cosine = cosine_similarity(final_movie)
np.fill_diagonal(cosine, 0 )
similarity_with_movie = pd.DataFrame(cosine,index=final_movie.index)
similarity_with_movie.columns=final_movie.index
similarity_with_movie.head()

userid,1,5,6,8,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,28,29,30,31,32,35,36,37,42,43,44,45,46,47,48,50,51,52,53,54,55,57,58,60,62,63,64,65,66,67,68,69,70,71,72,75,76,77,78,80,81,82,83,84,85,86,87,89,92,94,95,96,97,98,99,101,102,103,106,108,109,112,113,114,115,116,117,118,119,122,123,125,127,128,129,130,131,135,136,137,138,139,140,141,145,147,148,149,150,151,152,153,154,155,157,158,159,160,162,165,168,170,172,173,175,176,177,178,180,181,182,185,186,187,188,192,194,195,196,198,199,200,201,202,203,207,208,209,211,212,213,216,218,220,221,222,225,226,227,230,231,232,233,237,238,240,242,243,244,246,248,249,250,251,255,256,257,258,259,262,264,265,267,268,270,271,272,274,275,276,277,279,281,282,283,284,286,287,289,290,291,292,294,295,296,297,298,299,300,301,302,303,304,306,307,309,310,311,314,315,316,318,319,320,321,323,324,326,328,330,331,336,337,338,340,342,343,344,345,346,347,349,350,351,352,355,357,360,361,362,364,367,370,371,372,373,374,376,378,379,380,382,385,386,389,390,391,394,395,396,397,398,399,400,402,403,404,405,406,407,409,412,413,414,415,416,417,419,422,424,425,426,427,428,429,430,431,433,435,437,438,439,442,443,446,447,449,450,451,452,453,454,455,456,457,459,460,461,463,464,465,467,468,469,470,472,473,476,477,479,481,483,484,486,487,490,492,493,494,496,497,498,499,500,501,502,503,506,508,509,513,514,517,518,519,520,521,522,523,524,525,526,527,528,529,530,532,533,534,535,536,537,538,539,540,541,543,544,545,546,549,550,552,553,554,555,556,557,558,560,561,562,563,566,567,568,569,570,571,572,573,575,577,579,580,581,582,583,584,586,589,590,592,593,594,596,597,599,601,604,605,606,607,608,612,614,615,617,619,621,622,624,626,627,628,629,630,632,633,634,635,637,638,640,641,642,647,648,651,652,653,657,658,661,662,664,665,666,668,670,671,672,677,678,679,680,681,684,685,686,687,688,689,690,691,692,693,694,697,699,700,702,703,705,706,707,708,709,710,711,712,715,716,717,718,719,721,723,724,725,727,728,730,731,732,733,735,736,737,738,740,742,743,744,745,746,748,749,750,754,755,756,758,759,760,761,762,763,764,765,768,769,770,771,772,777,779,781,783,785,786,788,789,790,791,792,793,795,797,798,799,802,803,804,805,806,808,810,812,813,814,815,817,819,821,823,824,825,826,827,829,833,836,837,839,843,844,845,846,847,849,850,851,853,854,855,857,858,859,860,861,862,864,865,867,868,869,870,875,876,877,878,880,881,882,884,885,886,887,888,889,890,891,893,894,895,897,899,900,901,902,904,905,906,907,908,910,912,913,916,917,919,920,921,923,924,925,927,928,929,930,933,935,937,939,940,941,943
userid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.777237,0.785027,0.864183,0.825385,0.852909,0.648198,0.838233,0.788096,0.814880,0.872719,0.763039,0.861970,0.834435,0.766491,0.815426,0.823734,0.849473,0.856361,0.851505,0.869350,0.861096,0.857585,0.868323,0.855025,0.844223,0.848675,0.799704,0.773996,0.820477,0.868652,0.864470,0.868506,0.841103,0.845857,0.862541,0.862198,0.866254,0.846432,0.861514,0.844232,0.796165,0.847074,0.784083,0.836238,0.868929,0.846200,0.862758,0.863680,0.845061,0.856542,0.844590,0.845672,0.785029,0.831324,0.834873,0.828414,0.853988,0.877895,0.845530,0.797936,0.826949,0.855119,0.792516,0.862023,0.806154,0.829783,0.778209,0.832324,0.783737,0.836662,0.828362,0.847806,0.814934,0.840138,0.867532,0.864178,0.856323,0.866868,0.742178,0.860509,0.841607,0.854623,0.821317,0.842546,0.859662,0.861637,0.819899,0.848054,0.844209,0.792800,0.831948

In [17]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [18]:
# top 30 neighbours for each user
#sim_user_30_u = find_n_neighbours(similarity_with_user,30)
#sim_user_30_u.head()

In [19]:
# top 30 neighbours for each user
sim_user_30_m = find_n_neighbours(similarity_with_movie,30)
sim_user_30_m.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,157,517,80,225,549,895,677,864,827,772,594,319,888,876,849,687,728,691,400,231,17,571,628,754,810,920,376,769,685,662
5,157,517,677,584,80,728,319,571,772,231,864,849,549,888,827,102,563,810,895,165,876,687,355,513,400,431,140,414,855,594
6,157,517,647,376,594,596,849,652,677,700,513,80,319,419,876,573,139,225,571,772,855,781,920,744,685,888,827,477,723,544
8,517,157,772,80,549,888,677,849,319,864,728,386,827,594,400,231,876,376,810,687,477,431,855,355,785,941,723,920,558,584
10,517,157,80,319,677,849,888,864,571,594,583,728,231,855,876,549,596,558,772,810,242,400,700,827,941,477,688,386,431,687


In [20]:
#def get_user_similar_movies( user1, user2 ):
 #   common_movies = Rating_avg[Rating_avg.userid == user1].merge(
  #  Rating_avg[Rating_avg.userid == user2],
   # on = "itemid",
    #how = "inner" )
    #return common_movies.merge( movies, on = 'itemid' )

In [21]:
#a = get_user_similar_movies(1,2)
#a = a.loc[ : , ['rating_x_x','rating_x_y','movietitle']]
#a.head()

,rating_x_x,rating_x_y,movietitle


In [22]:
#def User_item_score(user,item):
    #a = sim_user_30_m[sim_user_30_m.index==user].values
    #b = a.squeeze().tolist()
    #c = final_movie.loc[:,item]
    #d = c[c.index.isin(b)]
    #f = d[d.notnull()]
    #avg_user = Mean.loc[Mean['userid'] == user,'rating'].values[0]
    #index = f.index.values.squeeze().tolist()
    #corr = similarity_with_movie.loc[user,index]
    #fin = pd.concat([f, corr], axis=1)
    #fin.columns = ['adg_score','correlation']
    #fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    #nume = fin['score'].sum()
    #deno = fin['correlation'].sum()
    #final_score = avg_user + (nume/deno)
    #return final_score

In [23]:
#score = User_item_score(1,2)
#print("score (u,i) is",score)

In [24]:
#Rating_avg = Rating_avg.astype({"itemid": str})
Movie_user = Rating_avg.groupby(by = 'userid')['itemid'].apply(lambda x:','.join(x))

In [30]:
def User_item_score1(user):
    Movie_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = Movie_user[Movie_user.index.isin(b)]
    l = ','.join(d.values)
    Movie_seen_by_similar_users = l.split(',')
    Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
    Movies_under_consideration = list(map(int, Movies_under_consideration))
    score = []
    for item in Movies_under_consideration:
        c = final_movie.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['userid'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_movie.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'itemid':Movies_under_consideration,'score':score})
    top_10_recommendation = data.sort_values(by='score',ascending=False).head(10)
    Movie_Name = top_10_recommendation.merge(movies, how='inner', on='itemid')
    Movie_Names = Movie_Name.movietitle.values.tolist()
    return Movie_Names

In [51]:
user = int(input("Enter the user id to whom you want to recommend : "))
predicted_movies = User_item_score1(user)
print(" ")
print("The Recommendations for your chosen UserID")
print("   ")
for i in predicted_movies:
    print(i)

Enter the user id to whom you want to recommend : 13
 
The Recommendations for your chosen UserID
   
Nânette et Boni (1996)
Close Shave, A (1995)
Wallace & Gromit: The Best of Aardman Animation (1996)
Chasing Amy (1997)
Hard Eight (1996)
Hugo Pool (1997)
Full Monty, The (1997)
Grand Day Out, A (1992)
Wings of Desire (1987)
When We Were Kings (1996)


In [50]:
Ratings_import[Ratings_import['userid'] == 13][Ratings_import['group'] == 'train'].sort_values(['group','itemid'])

#Full Monty is number 269 user13


/Users/kennyliang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,userid,itemid,rating,timestamp,date,group
30881,13,1,3,882140487,1997-12-14,train
4470,13,2,3,882397650,1997-12-17,train
21009,13,4,5,882141306,1997-12-14,train
39066,13,5,1,882396869,1997-12-17,train
34107,13,7,2,882396790,1997-12-17,train
92935,13,8,4,882140001,1997-12-14,train
91258,13,9,3,882140205,1997-12-14,train
19603,13,11,1,882397146,1997-12-17,train
33061,13,12,5,881515011,1997-12-07,train
31913,13,13,5,882141617,1997-12-14,train


In [37]:
item[item['itemid'] == ]

,movieid,movietitle,releasedate,imdburl,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,filmnoir,horror,musical,mystery,romance,scifi,thriller,war,western
0,1,Toy Story (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
1,2,GoldenEye (1995),1-Jan-95,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0
2,3,Four Rooms (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0
3,4,Get Shorty (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0
4,5,Copycat (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0.0,0.0
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,1-Jan-95,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0
6,7,Twelve Monkeys (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0.0,0.0
7,8,Babe (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0
8,9,Dead Man Walking (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Dead%20Man%20...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0
9,10,Richard III (1995),22-Jan-96,http://us.imdb.com/M/title-exact?Richard%20III...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0


# CF Predicting Rating

In [67]:
from surprise import Reader, Dataset
from surprise import SVD, evaluate
from surprise import NMF

reader = Reader(rating_scale=(0.0, 5.0))
ratings = Ratings 
data = Dataset.load_from_df(ratings[['userid', 'itemid', 'rating']], reader)

In [78]:
data.split(n_folds=5)

algo = SVD()
evaluate(algo, data, measures=['RMSE'])


/Users/kennyliang/anaconda3/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Users/kennyliang/anaconda3/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.9446
------------
Fold 2
RMSE: 0.9382
------------
Fold 3
RMSE: 0.9403
------------
Fold 4
RMSE: 0.9322
------------
Fold 5
RMSE: 0.9444
------------
------------
Mean RMSE: 0.9399
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.9445788788738093,
                             0.9382161578185835,
                             0.9403085747027707,
                             0.9322335053566436,
                             0.9443507023165056]})

In [70]:
algo = SVD()
trainset = data.build_full_trainset()
algo.fit(trainset)

In [76]:
algo.predict(1, 1)

Prediction(uid=1, iid=1, r_ui=None, est=4.282207059988077, details={'was_impossible': False})

In [75]:
item[item['movieid'] == 1]

,movieid,movietitle,releasedate,imdburl,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,filmnoir,horror,musical,mystery,romance,scifi,thriller,war,western
0,1,Toy Story (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0


# CF Test 2

In [71]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(data[['userid', 'movieid', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()
algo.fit(trainset)

In [72]:
test = algo.test(testset)
test = pd.DataFrame(test)
test.drop("details", inplace=True, axis=1)
test.columns = ['userid', 'movieid', 'actual', 'prediction']

In [73]:
print('The RMSE of CF Filtering is ' + str(recmetrics.rmse(test.actual, test.prediction)))

The RMSE of CF Filtering is 0.934427262722221


In [66]:
cf_model = test.pivot_table(index='userid', columns='movieid', values='prediction').fillna(0)

In [67]:
def get_users_predictions(user_id, n, model):
    recommended_items = pd.DataFrame(model.loc[user_id])
    recommended_items.columns = ["predicted_rating"]
    recommended_items = recommended_items.sort_values('predicted_rating', ascending=False)    
    recommended_items = recommended_items.head(n)
    return recommended_items.index.tolist()

In [68]:
get_users_predictions(156, 10, cf_model)

[480, 192, 357, 11, 651, 58, 157, 806, 1, 990]

In [79]:
#format test data
test = test.copy().groupby('userid')['movieid'].agg({'actual': (lambda x: list(set(x)))})
#make recommendations for all members in the test data
recs = []
for user in test.index:
    prediction = get_users_predictions(user, 10, cf_model)
    recs.append(prediction)
        
test['prediction'] = recs
test.head()

/Users/kennyliang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,actual,prediction
userid,,
1,"[256, 1, 128, 259, 131, 139, 267, 14, 143, 16,...","[179, 199, 223, 209, 48, 190, 7, 197, 257, 262]"
2,"[258, 291, 100, 293, 297, 314, 302, 303, 306, ...","[272, 1, 10, 258, 311, 14, 292, 273, 257, 288]"
3,"[353, 260, 325, 327, 330, 299, 268, 332, 334, ...","[302, 320, 317, 327, 347, 337, 322, 329, 343, ..."
4,"[260, 356, 294, 359, 264, 329, 361, 11, 300, 3...","[288, 271, 327, 356, 358, 1, 989, 997, 996, 995]"
5,"[1, 386, 389, 390, 393, 396, 400, 144, 403, 40...","[50, 169, 430, 194, 189, 186, 434, 211, 222, 432]"


In [80]:
actual = test.actual.values.tolist()
prediction = test.prediction.values.tolist()

cf_recall = []
for K in np.arange(1,11):
    cf_recall.extend([recmetrics.mark(actual, prediction, k=K)])
cf_recall

[0.019298034208381123,
 0.031786969592093456,
 0.0398617432735463,
 0.0477321599134852,
 0.0546462686663862,
 0.05939120657545153,
 0.06399550129350724,
 0.06763224127950435,
 0.07048869567922793,
 0.07289667082518429]